In [131]:
import torch
torchversion = torch.__version__

In [132]:
torchversion

'2.0.0+cu118'

In [133]:
# Install PyTorch Scatter, PyTorch Sparse, and PyTorch Geometric
!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-{torchversion}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Visualization
import networkx as nx
import matplotlib.pyplot as plt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [134]:
#@title Load Dataset - ENZMES
from torch_geometric.datasets import TUDataset, GNNBenchmarkDataset, Planetoid

dataset = TUDataset(root='.', name='ENZYMES').shuffle()
# dataset = GNNBenchmarkDataset(root='.', name='MNIST').shuffle()   # The large dataset MINIST 100 epoches take over 8 hours

# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {dataset[0].x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: ENZYMES(600)
-------------------
Number of graphs: 600
Number of nodes: 18
Number of features: 3
Number of classes: 6


In [135]:
#@title Data split
from torch_geometric.loader import DataLoader

# Create training, validation, and test sets
train_dataset = dataset[:int(len(dataset)*0.8)]
val_dataset   = dataset[int(len(dataset)*0.8):int(len(dataset)*0.9)]
test_dataset  = dataset[int(len(dataset)*0.9):]

print(f'Training set   = {len(train_dataset)} graphs')
print(f'Validation set = {len(val_dataset)} graphs')
print(f'Test set       = {len(test_dataset)} graphs')

# Create mini-batches
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print('\nTrain loader:')
for i, subgraph in enumerate(train_loader):
    print(f' - Subgraph {i}: {subgraph}')

print('\nValidation loader:')
for i, subgraph in enumerate(val_loader):
    print(f' - Subgraph {i}: {subgraph}')

print('\nTest loader:')
for i, subgraph in enumerate(test_loader):
    print(f' - Subgraph {i}: {subgraph}')

Training set   = 480 graphs
Validation set = 60 graphs
Test set       = 60 graphs

Train loader:
 - Subgraph 0: DataBatch(edge_index=[2, 3802], x=[977, 3], y=[32], batch=[977], ptr=[33])
 - Subgraph 1: DataBatch(edge_index=[2, 4162], x=[1063, 3], y=[32], batch=[1063], ptr=[33])
 - Subgraph 2: DataBatch(edge_index=[2, 4136], x=[1141, 3], y=[32], batch=[1141], ptr=[33])
 - Subgraph 3: DataBatch(edge_index=[2, 3656], x=[949, 3], y=[32], batch=[949], ptr=[33])
 - Subgraph 4: DataBatch(edge_index=[2, 3782], x=[1059, 3], y=[32], batch=[1059], ptr=[33])
 - Subgraph 5: DataBatch(edge_index=[2, 3968], x=[1090, 3], y=[32], batch=[1090], ptr=[33])
 - Subgraph 6: DataBatch(edge_index=[2, 3830], x=[981, 3], y=[32], batch=[981], ptr=[33])
 - Subgraph 7: DataBatch(edge_index=[2, 4240], x=[1087, 3], y=[32], batch=[1087], ptr=[33])
 - Subgraph 8: DataBatch(edge_index=[2, 4058], x=[1077, 3], y=[32], batch=[1077], ptr=[33])
 - Subgraph 9: DataBatch(edge_index=[2, 3942], x=[1010, 3], y=[32], batch=[1010],

In [136]:
#@title GNNs
from torch.nn import Linear, Sequential, BatchNorm1d, ReLU, Dropout, LeakyReLU
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphSAGE, GINConv, SAGEConv
from torch_geometric.nn import global_mean_pool, global_add_pool
from torch_scatter import scatter_add


class GCN(torch.nn.Module):
    def __init__(self, dim_h):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, dim_h)
        self.conv2 = GCNConv(dim_h, dim_h)
        self.lin = Linear(dim_h, dataset.num_classes)

    def forward(self, x, edge_index, batch): 
        h = self.conv1(x, edge_index)
        h = h.relu()
        h = self.conv2(h, edge_index)
        h = h.relu()

        hG = global_mean_pool(h, batch)

        h = F.dropout(hG, p=0.2, training=self.training)
        h = self.lin(h)
        
        return hG, F.log_softmax(h, dim=1)

class GraphSAGENet(torch.nn.Module):
    def __init__(self, dim_h):
        super(GraphSAGENet, self).__init__()
        self.conv1 = SAGEConv(dataset.num_node_features, dim_h, aggr='mean')
        self.conv2 = SAGEConv(dim_h, dim_h, aggr='mean')
        self.lin = Linear(dim_h, dataset.num_classes)

    def forward(self, x, edge_index, batch):
        h = self.conv1(x, edge_index)
        h = h.relu()
        h = self.conv2(h, edge_index)
        h = h.relu()

        hG = global_mean_pool(h, batch)

        h = F.dropout(hG, p=0.2, training=self.training)
        h = self.lin(h)
        
        return hG, torch.log(F.softmax(h, dim=1))

class GIN(torch.nn.Module):
    def __init__(self, dim_h, num_layers=5):
        super(GIN, self).__init__()
        self.num_layers = num_layers
        self.mlps = torch.nn.ModuleList()
        for i in range(num_layers):
            if i == 0:
                self.mlps.append(Sequential(Linear(dataset.num_node_features, dim_h),
                                            BatchNorm1d(dim_h), LeakyReLU(),
                                            Linear(dim_h, dim_h), LeakyReLU()))
            else:
                self.mlps.append(Sequential(Linear(dim_h, dim_h),
                                            BatchNorm1d(dim_h), LeakyReLU(),
                                            Linear(dim_h, dim_h), LeakyReLU()))
        self.lin1 = Linear(dim_h*num_layers, dim_h*num_layers)
        self.lin2 = Linear(dim_h*num_layers, dim_h*num_layers)
        self.lin3 = Linear(dim_h*num_layers, dataset.num_classes)

    def forward(self, x, edge_index, batch):

        h_list = []
        for i in range(self.num_layers):
            if i == 0:
                h_list.append(self.mlps[i](x))
            else:
                h_list.append(self.mlps[i](h_list[i-1]))

        h_list = [scatter_add(h, batch, dim=0) for h in h_list]
        
        h = torch.cat(h_list, dim=1)

        h = self.lin1(h)
        h = F.leaky_relu(h, negative_slope=0.1)
        h = F.dropout(h, p=0.2, training=self.training)
        h = self.lin2(h)
        h = F.leaky_relu(h, negative_slope=0.1)
        h = F.dropout(h, p=0.2, training=self.training)
        h = self.lin3(h)
        return h, torch.log(F.softmax(h, dim=1))

gcn = GCN(dim_h=32)
graphsagenet = GraphSAGENet(dim_h=32)
gin = GIN(dim_h=32)

In [137]:
#@title Train models
def train(model, loader):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                      lr=0.01,
                                      weight_decay=0.01)
    epochs = 100

    model.train()
    for epoch in range(epochs+1):
      total_loss = 0
      acc = 0
      val_loss = 0
      val_acc = 0

      # Train on batches
      for data in loader:
        optimizer.zero_grad()
        _, out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y)
        total_loss += loss / len(loader)
        acc += accuracy(out.argmax(dim=1), data.y) / len(loader)
        loss.backward()
        optimizer.step()

        # Validation
        val_loss, val_acc = test(model, val_loader)

    # Print metrics every 10 epochs
    if(epoch % 10 == 0):
        print(f'Epoch {epoch:>3} | Train Loss: {total_loss:.2f} '
              f'| Train Acc: {acc*100:>5.2f}% '
              f'| Val Loss: {val_loss:.2f} '
              f'| Val Acc: {val_acc*100:.2f}%')
          
    test_loss, test_acc = test(model, test_loader)
    print(f'Test Loss: {test_loss:.2f} | Test Acc: {test_acc*100:.2f}%')
    
    return model

def test(model, loader):
    criterion = torch.nn.CrossEntropyLoss()
    model.eval()
    loss = 0
    acc = 0

    for data in loader:
      _, out = model(data.x, data.edge_index, data.batch)
      loss += criterion(out, data.y) / len(loader)
      acc += accuracy(out.argmax(dim=1), data.y) / len(loader)

    return loss, acc

def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

gcn = train(gcn, train_loader)
graphsagenet = train(graphsagenet, train_loader)
gin = train(gin, train_loader)

Epoch 100 | Train Loss: 1.79 | Train Acc: 17.29% | Val Loss: 1.79 | Val Acc: 22.32%
Test Loss: 1.80 | Test Acc: 6.47%
Epoch 100 | Train Loss: 1.78 | Train Acc: 22.29% | Val Loss: 1.77 | Val Acc: 26.56%
Test Loss: 1.80 | Test Acc: 8.04%
Epoch 100 | Train Loss: 1.76 | Train Acc: 21.25% | Val Loss: 1.79 | Val Acc: 20.09%
Test Loss: 1.75 | Test Acc: 21.88%


In [138]:
#@title Test and compare - GIN outperformed GCN and GraphSAGE
gcn.eval()
graphsagenet.eval()
gin.eval()
acc_gcn = 0
acc_graphSAGE = 0
acc_gin = 0

for data in test_loader:
    # Get classifications
    _, out_gcn = gcn(data.x, data.edge_index, data.batch)
    _, out_graphSAGE = graphsagenet(data.x, data.edge_index, data.batch)
    _, out_gin = gin(data.x, data.edge_index, data.batch)

    # Calculate accuracy scores
    acc_gcn += accuracy(out_gcn.argmax(dim=1), data.y) / len(test_loader)
    acc_graphSAGE += accuracy(out_graphSAGE.argmax(dim=1), data.y) / len(test_loader)
    acc_gin += accuracy(out_gin.argmax(dim=1), data.y) / len(test_loader)

# Print results
print(f'GCN accuracy:     {acc_gcn*100:.2f}%')
print(f'GraphSAGE accuracy:     {acc_graphSAGE*100:.2f}%')
print(f'GIN accuracy:     {acc_gin*100:.2f}%')

GCN accuracy:     6.47%
GraphSAGE accuracy:     8.04%
GIN accuracy:     21.88%
